# FHM Model

## Sources
- [] On Veracity of Macroscopic Lithium-Ion Battery Models  10.1149/2.0771509jes
- [] What if the Doyle-Fuller-Newman Model Fails? A New Macroscale Modeling Framework 10.1109/CDC.2018.8619793

- [] Simplification of full homogenized macro-scale model for lithium-ion batteries  10.1016/j.est.2021.103801

## Setup

In [2]:
import sys
print(sys.version)

3.10.11 (main, Apr  5 2023, 00:00:00) [GCC 13.0.1 20230404 (Red Hat 13.0.1-0)]


In [3]:
%%capture
! python3.10 -m pip install --upgrade pip
! python3.10 -m pip install numpy
! python3.10 -m pip install matplotlib
! python3.10 -m pip install scikit-fem

In [4]:
! python3.10 -m pip install julia
import julia
julia.install()

Defaulting to user installation because normal site-packages is not writeable


[ Info: Julia version info


Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
      "Fedora release 38 (Thirty Eight)"
  uname: Linux 6.2.11-300.fc38.x86_64 #1 SMP PREEMPT_DYNAMIC Thu Apr 13 20:27:09 UTC 2023 x86_64 unknown
  CPU: AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx: 
              speed         user         nice          sys         idle          irq
       #1  2398 MHz       2045 s         40 s        568 s      12068 s        219 s
       #2  2399 MHz       2023 s         34 s        489 s      12369 s         81 s
       #3  1199 MHz       2131 s         21 s        543 s      12176 s         92 s
       #4  1400 MHz       2049 s         44 s        458 s      12387 s         81 s
       #5  1198 MHz       2156 s         23 s        499 s      12203 s         97 s
       #6  1200 MHz       2081 s         24 s        434 s      12394 s         82 s
       #7  2349 MHz       1875 s         20 s        535 s      12468 s         87 s
       #8  

[ Info: Julia executable: /home/willi/.julia/juliaup/julia-1.8.5+0.x64.linux.gnu/bin/julia
[ Info: Trying to import PyCall...
┌ Info: PyCall is already installed and compatible with Python executable.
│ 
│ PyCall:
│     python: /usr/bin/python3.10
│     libpython: /usr/lib64/libpython3.10.so.1.0
│ Python:
│     python: /usr/bin/python3.10
└     libpython: /usr/lib64/libpython3.10.so.1.0


In [5]:
! python3.10 -m pip install diffeqpy

Defaulting to user installation because normal site-packages is not writeable


## Model-implementation

### FEM

In [ ]:
import numpy as np
from skfem import MeshLine, Basis, ElementLineP2, BilinearForm, LinearForm
from skfem.helpers import dot, grad


# SIMULATION CONTROL
n_points_particle = 20 + 1
# MESHING
space = np.linspace(0, 1, n_points_particle)
mesh = MeshLine(space).with_boundaries(
    {"inner": lambda xi: xi[0] == 0, "surface": lambda xi: xi[0] == 1.0}
)

basis = Basis(mesh, ElementLineP2())  # ElementLineHermite
basis_surf = basis.boundary("surface")
dofs = basis.get_dofs("surface")